In [1]:
import pandas as pd
import os, glob
from owlready2 import *

import owlready2
print(owlready2.VERSION) ## Bugs when > 0.40 but works file at owlready2==0.40 

#```Merging ontologies:
#https://stackoverflow.com/questions/75102102/merge-two-ontologies-using-owlready
#```

0.40


In [5]:
def debug(onto):
    print("\n# Overview\n")
    for k in onto.classes():
        if len(k.instances()):
            print("* ","_"+str(k).split(".")[-1]+"_"," -- ",len(k.instances()), "instances.")

In [6]:
onto = get_ontology("../kg/pbn_t3_5.owl").load()
debug(onto)


# Overview

*  _PBNThing_  --  19210 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  376 instances.
*  _Risk_  --  3418 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2640 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  3915 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  5295 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _Blueprint_  --  24 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _BP_Category_  --  3 instances.
*  _BP_Theme_  --  10 instances.
*  _PBNCategory_  --  330 instances.


In [4]:
onto.save("../kg/new/pbn_t3_5_noPBN.owl")